In [1]:
import modin.pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import eda_functions
%matplotlib inline

### EDA preprocessing

In [2]:
PATH = '../data/eda.csv'
df = eda_functions.eda_preprocessing(PATH)
df.head()

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target,blue_range,blue_iqr,blue_skew,blue_kurtosis,green_range,green_iqr,green_skew,green_kurtosis,red_range,red_iqr,red_skew,red_kurtosis
0,ISIC_2637011,IP_7279968,male,45.0,head/neck,unknown,benign,0,285.0,98.50,0.851258,0.505571,285.0,98.50,0.851258,0.505571,285.0,98.50,0.851258,0.505571
1,ISIC_0015719,IP_3075186,female,45.0,upper extremity,unknown,benign,0,225.0,135.25,0.529849,-1.069920,225.0,135.25,0.529849,-1.069920,225.0,135.25,0.529849,-1.069920
2,ISIC_0052212,IP_2842074,female,50.0,lower extremity,nevus,benign,0,125.0,41.00,1.313852,0.491882,125.0,41.00,1.313852,0.491882,125.0,41.00,1.313852,0.491882
3,ISIC_0068279,IP_6890425,female,45.0,head/neck,unknown,benign,0,111.0,29.00,1.372237,1.365879,111.0,29.00,1.372237,1.365879,111.0,29.00,1.372237,1.365879
4,ISIC_0074268,IP_8723313,female,55.0,upper extremity,unknown,benign,0,339.0,107.25,1.123784,-0.064303,339.0,107.25,1.123784,-0.064303,339.0,107.25,1.123784,-0.064303
